In [1]:
from task2_utils import *
warnings.filterwarnings('ignore')
# interactive plot
%matplotlib widget

In [2]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")

In [3]:
def calculate_image_similarity(data, distance_measure):
    """Object-object similarity with given distance measure"""
    n = data.shape[0]
    image_sim_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            image_sim_matrix[i][j] = image_sim_matrix[j][i] = distance_measure(
                np.array(data[i]), np.array(data[j])
            )
    return image_sim_matrix

In [4]:
def mds_projection(data_sim_matrix, n_components=2):
    """MDS projection to n-D space"""
    n = data_sim_matrix.shape[0]
    # Centering matrix
    C = np.eye(n) - np.ones((n, n)) / n
    # B = -1/2 * C * D^2 * C
    B = -0.5 * C @ (data_sim_matrix**2) @ C
    # Eigen decomposition
    eigvals, eigvecs = np.linalg.eigh(B)

    # Sort eigenvalues and corresponding eigenvectors
    indices = np.argsort(eigvals)[::-1]
    eigvals = eigvals[indices]
    eigvecs = eigvecs[:, indices]

    # Take the first n_components eigenvectors
    components = eigvecs[:, :n_components]

    return components

In [5]:
def avgandmin_knn_distance(data_sim_matrix, k):
    """Get avg. and minimum k-th nearest neighbor distance"""
	# Sort each row of the distance matrix and extract the kth-nearest neighbor distance
    kth_neighbor_distances = np.sort(data_sim_matrix, axis=1)[:, k-1]

    # Understanding KNN distribution to figure out strategy to find epsilon range
    # plt.plot(np.sort(kth_neighbor_distances))
    # plt.show()
    
    # Calculate the average and minimum distance of the kth-nearest neighbor
    average_distance = np.mean(kth_neighbor_distances)
    minimum_distance = np.min(kth_neighbor_distances)

    return average_distance, minimum_distance


In [6]:
def display_cluster_stats(clusters):
    """Display cluster counts and noise point count"""
    cluster_counts = np.unique(clusters, return_counts=True)
    cluster_counts_dict = dict(
        (unique_label, unique_count)
        for unique_label, unique_count in zip(cluster_counts[0], cluster_counts[1])
    )
    print("Clusters:", cluster_counts_dict)
    print("No. of clusters:", len(cluster_counts_dict.keys() - {-1}))
    if -1 in cluster_counts_dict:
        print("Noise points:", cluster_counts_dict[-1])
    else:
        print("No noise points")

In [7]:
class DBSCAN:
    def __init__(
        self, label, data, distance_measure, eps, min_samples, data_sim_matrix=None
    ):
        self.label = label
        self.eps = eps
        self.min_samples = min_samples

        self.data = data
        self.distance_measure = distance_measure
        self.num_images = data.shape[0]

        self.image_sim_matrix = np.zeros((self.num_images, self.num_images))
        if data_sim_matrix is not None:
            self.image_sim_matrix = data_sim_matrix

        self.clusters = np.zeros(self.num_images)  # 0 represents unclassified points
        self.core_points = []

    def dbscan(self):
        """DBSCAN algorithm"""
        # if similarities not provided/calculated already
        if np.array_equal(
            self.image_sim_matrix, np.zeros((self.num_images, self.num_images))
        ):
            calculate_image_similarity(self.data, self.distance_measure)

        cluster_id = 0
        for i in range(self.num_images):
            if self.clusters[i] != 0:
                continue  # Skip already classified points

            neighbors = self.region_query(i)
            if len(neighbors) < self.min_samples:
                self.clusters[i] = -1  # Mark point as noise
            else:
                cluster_id += 1  # New cluster identified
                self.clusters[i] = cluster_id
                self.grow_cluster(neighbors, cluster_id)

        return self.clusters

    def region_query(self, center):
        distances = self.image_sim_matrix[center]
        return [i for i, dist in enumerate(distances) if dist < self.eps]

    def grow_cluster(self, neighbors, cluster_id):
        i = 0
        # check neighbors for connected components
        while i < len(neighbors):
            neighbor = neighbors[i]

            if self.clusters[neighbor] == -1:
                self.clusters[neighbor] = cluster_id  # Change noise to border point
            elif self.clusters[neighbor] == 0:
                self.clusters[neighbor] = cluster_id
                new_neighbors = self.region_query(neighbor)
                # If new point is a core point
                if len(new_neighbors) >= self.min_samples:
                    neighbors += new_neighbors  # add its neighbors to list of neighbors to consider
            i += 1
    
    def get_core_points(self, label_img_ids):
        """Find core points (after clustering only!)"""
        for i in range(self.num_images):
            if self.clusters[i] == -1:
                continue    # Skip noise points

            neighbors = self.region_query(i)
            if len(neighbors) < self.min_samples:
                continue    # not a core point
            else:
                self.core_points.append(label_img_ids[i])

    def mds_scatter_clusters(self):
        """Visualize clusters as point clouds in 2-D space"""
        # Perform MDS projection
        mds_components = mds_projection(self.image_sim_matrix)

        # Plot clusters
        plt.figure(figsize=(8, 6))
        for label in set(self.clusters):
            cluster_points = mds_components[self.clusters == label]
            plt.scatter(
                cluster_points[:, 0],
                cluster_points[:, 1],
                label=f"{(f'Cluster {int(label)}') if label != -1 else 'Noise points'}",
            )

        plt.title("DBSCAN clusters projected onto 2-D MDS space")
        plt.xlabel("MDS component 1")
        plt.ylabel("MDS component 2")
        plt.legend()
        plt.savefig(f"Plots/DBSCAN_MDS_Label_{self.label}.png")
        plt.show()

    def group_image_clusters(self, image_data):
        # Perform MDS projection
        mds_components = mds_projection(self.image_sim_matrix)
        # Scaling up to fit images inside
        mds_components = mds_components * 10000

        min_x_mds = np.min(mds_components[:, 0])
        min_y_mds = np.min(mds_components[:, 1])
        max_x_mds = np.max(mds_components[:, 0])
        max_y_mds = np.max(mds_components[:, 1])

        img_width = (max_x_mds - min_x_mds) / 10
        img_height = (max_y_mds - min_y_mds) / 10

        # Plot clusters
        plt.figure(figsize=(8, 6))
        for label in set(self.clusters):
            cluster_points = mds_components[self.clusters == label]
            plt.scatter(
                cluster_points[:, 0],
                cluster_points[:, 1],
                label=f"{(f'Cluster {int(label)}') if label != -1 else 'Noise points'}",
                zorder=1,
            )

            # Display image thumbnails at cluster centroids
            cluster_indices = np.where(self.clusters == label)[0]
            cluster_center = np.mean(mds_components[cluster_indices], axis=0)
            thumbnail_data = image_data[cluster_indices[0]].resize(
                (int(np.ceil(img_width)), int(np.ceil(img_height)))
            )
            im = plt.imshow(
                thumbnail_data,
                extent=(
                    cluster_center[0] - 0.5 * img_width,
                    cluster_center[0] + 0.5 * img_width,
                    cluster_center[1] - 0.5 * img_height,
                    cluster_center[1] + 0.5 * img_height,
                ),
                interpolation="nearest",
                cmap=plt.cm.gray_r,
                zorder=0,
            )

            # Image border
            x1, x2, y1, y2 = im.get_extent()
            (im_border,) = plt.plot(
                [x1, x2, x2, x1, x1],
                [y1, y1, y2, y2, y1],
                "-",
                linewidth=2,
                solid_capstyle="butt",
                zorder=0,
            )

            # Click to bring to front
            def region_click(event, region_area=im, region_border=im_border):
                if region_area.contains(event)[0]:
                    region_border.set_zorder(2)
                    region_area.set_zorder(2)
                else:
                    region_border.set_zorder(0)
                    region_area.set_zorder(0)

            im.figure.canvas.mpl_connect("button_press_event", region_click)

        plt.title("2-D MDS space with image thumbnails at centroids")
        plt.xlabel("MDS component 1")
        plt.ylabel("MDS component 2")
        ax = plt.gca()
        ax.margins(0.05)
        ax.set_aspect(0.75 / ax.get_data_ratio())
        plt.legend()
        plt.savefig(f"Plots/DBSCAN_MDS_Label_{self.label}_with_images.png")
        plt.show()

In [8]:
# selected_feature_model = valid_feature_models[
#     str(input("Enter feature model - one of " + str(list(valid_feature_models.keys()))))
# ]
selected_feature_model = valid_feature_models["avgpool"]
# selected_distance_measure = euclidean_distance_measure
selected_distance_measure = feature_distance_matches[selected_feature_model]
selected_c = 5

In [20]:
best_models = []
for label in range(NUM_LABELS):
# for label in [0, 1]:
    print("Clustering label", label, "...")
    # get label's images in PIL format
    label_imgs = []
    label_img_ids = [
        label_img["image_id"] for label_img in fd_collection.find({"true_label": label})
    ]
    for img_id in label_img_ids:
        img, true_label = dataset[img_id]
        label_imgs.append(transforms.ToPILImage()(img))

    # get image features
    label_fds = np.array(
        [
            np.array(img_fds[selected_feature_model]).flatten()
            for img_fds in fd_collection.find({"true_label": label})
        ]
    )

    label_dbscan_results = (np.zeros(label_fds.shape[0]), 0, 0)
    label_min_noise = label_fds.shape[0]
    label_min_cluster_diff = np.inf

    label_img_sim_matrix = calculate_image_similarity(
        label_fds, selected_distance_measure
    )

    # decrementally try min_samples, starting from twice the desired no. of clusters
    for cur_min_samples in range(2 * selected_c, 1, -1):
        # find range of epsilon to try, by checking all from mean to min knn distance
        # k is current min_samples
        max_eps, min_eps = avgandmin_knn_distance(label_img_sim_matrix, cur_min_samples)

        # try epsilon values
        for cur_eps in np.linspace(min_eps, max_eps, num=100):
            label_dbscan = DBSCAN(
                label,
                label_fds,
                selected_distance_measure,
                cur_eps,
                cur_min_samples,
                label_img_sim_matrix,
            )

            clusters = label_dbscan.dbscan()

            cluster_counts = np.unique(clusters, return_counts=True)
            cluster_counts_dict = dict(
                (unique_label, unique_count)
                for unique_label, unique_count in zip(
                    cluster_counts[0], cluster_counts[1]
                )
            )

            if cluster_counts_dict.get(-1) is not None:
                noise_pts = cluster_counts_dict.get(-1)
            else:
                noise_pts = 0
            cluster_diff = abs(len(cluster_counts_dict.keys() - {-1}) - selected_c)

            # store only most desirable clustering: as close as possible to c clusters, and then minimum noise
            if cluster_diff < label_min_cluster_diff or (
                cluster_diff == label_min_cluster_diff and noise_pts <= label_min_noise
            ):
                # print(
                #     "Better clustering:",
                #     label_dbscan_results[1],
                #     "->",
                #     cur_eps,
                #     "\t",
                #     label_dbscan_results[2],
                #     "->",
                #     cur_min_samples,
                # )
                # print(
                #     "Noise improvement:",
                #     label_min_noise,
                #     "->",
                #     noise_pts,
                #     "\tCluster count improvement:",
                #     label_min_cluster_diff,
                #     "->",
                #     cluster_diff,
                # )
                label_dbscan_results = (clusters, cur_eps, cur_min_samples)
                label_min_noise = noise_pts
                label_min_cluster_diff = cluster_diff

    best_label_dbscan = DBSCAN(
        label,
        label_fds,
        selected_distance_measure,
        label_dbscan_results[1],
        label_dbscan_results[2],
        label_img_sim_matrix,
    )
    best_label_dbscan.clusters = label_dbscan_results[0]
    best_label_dbscan.get_core_points(label_img_ids)

    # store best clustering
    best_models.append(best_label_dbscan)

    # # Interpretation
    # print("Label:", label)
    # # print("Epsilon:", label_dbscan_results[1], "\tMinPts:", label_dbscan_results[2])
    # display_cluster_stats(label_dbscan_results[0])
    # print("Core points:", len(best_label_dbscan.core_points))
    # # MDS point cloud
    # best_label_dbscan.mds_scatter_clusters()
    # # Image thumbnail overlay
    # best_label_dbscan.group_image_clusters(label_imgs)

Clustering label 0 ...
Clustering label 1 ...
Clustering label 2 ...
Clustering label 3 ...
Clustering label 4 ...
Clustering label 5 ...
Clustering label 6 ...
Clustering label 7 ...
Clustering label 8 ...
Clustering label 9 ...
Clustering label 10 ...
Clustering label 11 ...
Clustering label 12 ...
Clustering label 13 ...
Clustering label 14 ...
Clustering label 15 ...
Clustering label 16 ...
Clustering label 17 ...
Clustering label 18 ...
Clustering label 19 ...
Clustering label 20 ...
Clustering label 21 ...
Clustering label 22 ...
Clustering label 23 ...
Clustering label 24 ...
Clustering label 25 ...
Clustering label 26 ...
Clustering label 27 ...
Clustering label 28 ...
Clustering label 29 ...
Clustering label 30 ...
Clustering label 31 ...
Clustering label 32 ...
Clustering label 33 ...
Clustering label 34 ...
Clustering label 35 ...
Clustering label 36 ...
Clustering label 37 ...
Clustering label 38 ...
Clustering label 39 ...
Clustering label 40 ...
Clustering label 41 ...
Cl

Visualization:

In [37]:
for best_model in best_models:
    label_imgs = []
    label_img_ids = [
        label_img["image_id"]
        for label_img in fd_collection.find({"true_label": best_model.label})
    ]
    for img_id in label_img_ids:
        img, true_label = dataset[img_id]
        label_imgs.append(transforms.ToPILImage()(img))
    # Interpretation
    print("Label:", best_model.label)
    # print("Epsilon:", best_model.eps, "\tMinPts:", best_model.min_samples)
    display_cluster_stats(best_model.clusters)
    print("Core points:", len(best_model.core_points))
    # MDS point cloud
    best_model.mds_scatter_clusters()
    # # Image thumbnail overlay
    best_model.group_image_clusters(label_imgs)

Label: 0
Clusters: {-1.0: 94, 1.0: 15, 2.0: 94, 3.0: 3, 4.0: 4, 5.0: 8}
No. of clusters: 5
Noise points: 94
Core points: 103
Label: 1
Clusters: {-1.0: 32, 1.0: 154, 2.0: 11, 3.0: 4, 4.0: 8, 5.0: 8}
No. of clusters: 5
Noise points: 32
Core points: 106
Label: 2
Clusters: {-1.0: 60, 1.0: 4, 2.0: 4, 3.0: 8, 4.0: 4, 5.0: 20}
No. of clusters: 5
Noise points: 60
Core points: 35
Label: 3
Clusters: {-1.0: 162, 1.0: 218, 2.0: 3, 3.0: 3, 4.0: 9, 5.0: 4}
No. of clusters: 5
Noise points: 162
Core points: 199
Label: 4
Clusters: {-1.0: 16, 1.0: 9, 2.0: 3}
No. of clusters: 2
Noise points: 16
Core points: 12
Label: 5
Clusters: {-1.0: 166, 1.0: 222, 2.0: 2, 3.0: 2, 4.0: 5, 5.0: 3}
No. of clusters: 5
Noise points: 166
Core points: 234
Label: 6
Clusters: {-1.0: 10, 1.0: 7, 2.0: 2, 3.0: 2}
No. of clusters: 3
Noise points: 10
Core points: 11
Label: 7
Clusters: {-1.0: 15, 1.0: 4, 2.0: 2}
No. of clusters: 2
Noise points: 15
Core points: 6
Label: 8
Clusters: {-1.0: 12, 1.0: 9, 2.0: 2}
No. of clusters: 2
Noise 

In [21]:
full_fd_collection = getCollection("knravish_mwdb_phase_1", "fd_collection")

In [26]:
# Predict label based on nearest core point
all_core_pts = []
for best_model in best_models:
    all_core_pts.extend(best_model.core_points)
all_core_pts = [
    (x["image_id"], x["true_label"], np.array(x[selected_feature_model]))
    for x in full_fd_collection.find({"image_id": {"$in": all_core_pts}})
]

In [36]:
# all odd images
# for img_id in range(1, 8676, 2):
for img_id in [881]:
    img_fd = np.array(
        full_fd_collection.find_one({"image_id": img_id})[selected_feature_model]
    )
    distances = []
    for core_pt in all_core_pts:
        distances.append(
            (
                core_pt[0],
                core_pt[1],
                selected_distance_measure(
                    core_pt[2],
                    img_fd,
                ),
            )
        )
    print(sorted(distances, key=lambda dist: dist[2])[:selected_c])
    break

[(898, 2, 0.04471824055950302), (998, 2, 0.04670729369738935), (896, 2, 0.05814791090290394), (886, 2, 0.06548439873670464), (970, 2, 0.06915823780454072)]
